# VacationPy
----

In [1]:
# Uncomment and run if gmaps is not installed in your environment
# !pip install gmaps

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
city_data_df = pd.read_csv('../output_data/cities.csv', index_col='City_ID')
city_data_df.head()

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,jacareacanga,-6.22,-57.75,83.35,35,0,1.57,BR,1597444430
1,hamilton,39.18,-84.53,82.40,65,20,7.00,US,1597444251
2,hithadhoo,-0.60,73.08,82.04,75,89,8.63,MV,1597444260
3,chuy,-33.70,-53.46,52.59,92,76,6.04,UY,1597444434
4,punta arenas,-53.15,-70.92,33.80,80,90,4.70,CL,1597444435


### Humidity Heatmap
* Configure gmaps.
* Use the Latitude and Longitude as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [5]:
# Heatmap of humidity
locations = city_data_df[["Latitude", "Longitude"]]
humidity = city_data_df["Humidity"]
humidity_map = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=max(humidity), point_radius=5)

humidity_map.add_layer(heat_layer)

humidity_map

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#Ideal weather is max temp between 70 and 80 degrees, wind speed less than 10 mph, and zero cloudiness
condition1 = city_data_df['Max Temperature']>=70
condition2 = city_data_df['Max Temperature']<=80
condition3 = city_data_df['Wind Speed']<=10
condition4 = city_data_df['Cloudiness'] == 0

# Apply conditions and remove any Nan values
narrowed_city_df = city_data_df[(condition1&condition2&condition3&condition4)]
narrowed_city_df = narrowed_city_df.dropna()

# Display new data frame with only cities that match your conditions
narrowed_city_df

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
152,misratah,32.38,15.09,79.11,71,0,4.38,LY,1597444657
238,caravelas,-17.71,-39.25,71.80,84,0,7.00,BR,1597444808
277,marsa matruh,31.35,27.25,77.00,78,0,5.82,EG,1597444724
359,sabzevar,36.21,57.68,78.39,26,0,6.93,IR,1597444988
366,zitose,41.42,21.29,73.40,73,0,2.48,MK,1597444997
388,unai,-16.36,-46.91,73.20,40,0,3.60,BR,1597445030
407,salamiyah,35.01,37.05,70.32,69,0,7.83,SY,1597445056
438,xucheng,20.35,110.13,75.20,94,0,2.24,CN,1597445099
485,akdepe,42.06,59.38,73.40,43,0,6.93,TM,1597445165


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create a DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = narrowed_city_df[['City','Country','Latitude','Longitude']].copy()
hotel_df['Hotel Name'] = ""
hotel_df.head()

,City,Country,Latitude,Longitude,Hotel Name
City_ID,,,,,
152,misratah,LY,32.38,15.09,
238,caravelas,BR,-17.71,-39.25,
277,marsa matruh,EG,31.35,27.25,
359,sabzevar,IR,36.21,57.68,
366,zitose,MK,41.42,21.29,


In [8]:
# Create googleapis url and define parameters to search for a hotel
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "location": "lat,lng",
    "rankby": "distance",
    "keyword": "hotel",
    "key": g_key,
}

In [9]:
# Iterate through cities
for index, row in hotel_df.iterrows():

    # Get city, lat, and lng from hotel_df
    city = row['City']
    coords = f"{row['Latitude']},{row['Longitude']}"
    print(coords)
    # Set location in params dictionary
    params['location'] = coords

    # Assemble url and make API request
    print(f"Retrieving Results for {city}.")
    
    # Make an API request and convert response to JSON
    response = requests.get(base_url, params=params).json()
    
    # Extract results
    name_address = response['results']
    
    # Grab the first hotel from the results and the name, latitude, and longitude
    try:
        hotel_df.loc[index, 'Hotel Name'] = name_address[0]['name']
        hotel_df.loc[index, 'Latitude'] = name_address[0]['geometry']['location']['lat']
        hotel_df.loc[index, 'Longitude'] = name_address[0]['geometry']['location']['lng']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

hotel_df

32.38,15.09
Retrieving Results for misratah.
------------
-17.71,-39.25
Retrieving Results for caravelas.
------------
31.35,27.25
Retrieving Results for marsa matruh.
------------
36.21,57.68
Retrieving Results for sabzevar.
------------
41.42,21.29
Retrieving Results for zitose.
------------
-16.36,-46.91
Retrieving Results for unai.
------------
35.01,37.05
Retrieving Results for salamiyah.
------------
20.35,110.13
Retrieving Results for xucheng.
------------
42.06,59.38
Retrieving Results for akdepe.
Missing field/result... skipping.
------------
-9.66,120.26
Retrieving Results for waingapu.
------------
39.15,-3.02
Retrieving Results for tomelloso.
------------
-15.82,-50.61
Retrieving Results for itapirapua.
------------


,City,Country,Latitude,Longitude,Hotel Name
City_ID,,,,,
152,misratah,LY,32.380286,15.091802,الصفوة للشقق المفروشة
238,caravelas,BR,-17.735217,-39.264979,Pousada do Beco Shangri-La
277,marsa matruh,EG,31.353056,27.243252,Almanara Hotel Marsa Matrouh
359,sabzevar,IR,36.212394,57.675022,Farhangian Restaurant & Hotel
366,zitose,MK,41.370377,21.254396,Villa Papu
388,unai,BR,-16.358073,-46.905405,Hotel Brasília
407,salamiyah,SY,34.993645,37.057865,مسبح الخبازي
438,xucheng,CN,20.348242,110.151874,7 Days Inn
485,akdepe,TM,42.060000,59.380000,


In [10]:
# Filter out cities that don't return a hotel
condition = hotel_df['Hotel Name'] != ''
hotel_df = hotel_df[condition]
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
City_ID,,,,,
152,misratah,LY,32.380286,15.091802,الصفوة للشقق المفروشة
238,caravelas,BR,-17.735217,-39.264979,Pousada do Beco Shangri-La
277,marsa matruh,EG,31.353056,27.243252,Almanara Hotel Marsa Matrouh
359,sabzevar,IR,36.212394,57.675022,Farhangian Restaurant & Hotel
366,zitose,MK,41.370377,21.254396,Villa Papu
388,unai,BR,-16.358073,-46.905405,Hotel Brasília
407,salamiyah,SY,34.993645,37.057865,مسبح الخبازي
438,xucheng,CN,20.348242,110.151874,7 Days Inn
490,waingapu,ID,-9.654659,120.260994,Hotel Jemmy


In [11]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Latitude</dt><dd>{Latitude}</dd>
<dt>Longitude</dt><dd>{Longitude}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]
hotel_df 

,City,Country,Latitude,Longitude,Hotel Name
City_ID,,,,,
152,misratah,LY,32.380286,15.091802,الصفوة للشقق المفروشة
238,caravelas,BR,-17.735217,-39.264979,Pousada do Beco Shangri-La
277,marsa matruh,EG,31.353056,27.243252,Almanara Hotel Marsa Matrouh
359,sabzevar,IR,36.212394,57.675022,Farhangian Restaurant & Hotel
366,zitose,MK,41.370377,21.254396,Villa Papu
388,unai,BR,-16.358073,-46.905405,Hotel Brasília
407,salamiyah,SY,34.993645,37.057865,مسبح الخبازي
438,xucheng,CN,20.348242,110.151874,7 Days Inn
490,waingapu,ID,-9.654659,120.260994,Hotel Jemmy


In [12]:
# Add marker layer ontop of heat map
hotel_map = gmaps.figure()
hotel_map.add_layer(gmaps.marker_layer(locations,info_box_content=hotel_info,display_info_box=True))

# Display Map
hotel_map

Figure(layout=FigureLayout(height='420px'))